Netflix And YouTube Analysis, Last Part <3

1. ***Netflix Content Universe: An Animated Dashboard Exploration***

Uploading Datasets

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

netflix_df = pd.read_csv('netflix_titles.csv')

print(f'Dataset Shape: {netflix_df.shape}')
print('\nColumns:\n', netflix_df.columns)
netflix_df.head()

Basic Cleaning And KPI (Key Performance Indicator) Calculations

In [ ]:
import pandas as pd

#Converting 'date_added' to datetime
netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'], errors = 'coerce')


#Extracting year and month added for time-based analysis
netflix_df['year_added'] = netflix_df['date_added'].dt.year
netflix_df['month_added'] = netflix_df['date_added'].dt.month

#Cleaning duration column
#Duration is like '90 min' for movies or '3 Seasons' for TV shows
def extract_duration(val):
  if pd.isna(val):
    return None
  if 'Season' in val:
    return int(val.split()[0]) #number of seasons
  else:
      return int(val.split()[0]) #number of minutes

netflix_df['duration_num'] = netflix_df['duration'].apply(extract_duration)

#KPIs
total_titles = netflix_df.shape[0]
total_countries = netflix_df['country'].nunique()
total_genres = netflix_df['listed_in'].nunique()
total_types = netflix_df['type'].nunique()
avg_movie_duration = netflix_df[netflix_df['type'] == 'Movie']['duration_num'].mean()
avg_tv_seasons = netflix_df[netflix_df['type'] == 'TV Show']['duration_num'].mean()

#Displaying KPI(Key Performance Indicator/s)

print(f'Total Titles: {total_titles}')
print(f'Unique Counties: {total_countries}')
print(f'Unique Genres: {total_genres}')
print(f'Unique Types: {total_types}')
print(f'Average Movie Duration(min): {avg_movie_duration:.2f}')
print(f'Average TV Show Seasons: {avg_tv_seasons:.2f}')

Creating KPI cards with Plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Creating subplots with 3 columns to place KPI cards side by side
fig = make_subplots(
    rows = 2, cols = 3,
    specs = [[{'type':'indicator'}, {'type':'indicator'}, {'type':'indicator'}],
             [{'type':'indicator'}, {'type':'indicator'}, None]],
    subplot_titles = ('Total Titles', 'Unique Countries', 'Unique Genres',
                      'Content Types', 'Average Movie Duration (min)')

)
#Total Titles
fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_titles,
    title = {'text': 'Total Titles'},
    number = {'font': {'size': 30, 'color': 'darkblue'}}
), row = 1, col = 1)

#Unique Countries
fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_countries,
    title = {'text': 'Unique Coutries'},
    number = {'font': {'size': 30, 'color': 'darkgreen'}}
), row = 1, col = 2)

#Unique Genres
fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_genres,
    title = {'text': 'Total Genres'},
    number = {'font': {'size': 30, 'color': 'darkred'}}
), row = 1, col = 3)

#Content Types
fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_types,
    title = {'text': 'Content Type'},
    number = {'font': {'size': 30, 'color': 'purple'}}
), row = 2, col = 1)

#Average Movie Duration
fig.add_trace(go.Indicator(
    mode = 'number',
    value = round(avg_movie_duration, 2),
    title = {'text': 'Avg Movie Duration (min)'},
    number = {'font': {'size': 30, 'color': 'darkorange'}}
), row = 2, col = 2)

fig.update_layout(
    height = 400,
    showlegend = False,
    margin = dict(t = 50, b = 50, l = 50, r = 50),
    title_text = 'Netflix Key Performance Indicator (KPI) Overview',
    title_x = 0.5

)

fig.show()


Added the Bar Chart Race

In [ ]:
import pandas as pd
import plotly.express as px

#Filtering only valid year entries
df = netflix_df.dropna(subset = ['date_added', 'listed_in']).copy()

#Extracting year from 'date_added'
df['year_added'] = pd.to_datetime(df['date_added']).dt.year

#Creating 'genre_list by splitting 'listed_in'
df['genre_list'] = df['listed_in'].apply(lambda x: [i.strip() for i in x.split(',')])

#Extracting the first genre only to simply race
df['primary_genre'] = df['genre_list'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 'Unknown')

#Counting how many titles per genre per year
genre_yearly = df.groupby(['year_added','primary_genre']).size().reset_index(name = 'count')

#Sorting for better chart clarity
genre_yearly = genre_yearly.sort_values(by = 'year_added')

#Preview
print(genre_yearly.head())

In [ ]:
import plotly.express as px

fig_bar = px.bar(
    genre_yearly,
    x = 'count',
    y = 'primary_genre',
    color = 'primary_genre',
    orientation = 'h',
    animation_frame = 'year_added',
    range_x = [0, genre_yearly['count'].max() + 50],
    title = 'Netflix Genre Evolution (Bar Chart Race)',
    labels = {'primary_genre': 'Genre', 'count': 'Number of Titles'},
    template = 'plotly_dark',
    height = 600
)

fig_bar.update_layout(showlegend = False)
fig_bar.show()


Dashboard Creation

In [ ]:
import plotly.graph_objects as go

#Grouping data
titles_per_year = df.groupby('year_added').size().reset_index(name='total_titles')

#Creating initial trace (first year)
initial_trace = go.Indicator(
    mode = "number+delta",
    value = titles_per_year['total_titles'].iloc[0],
    delta = {'reference': 0, 'relative': True, 'position': "right",
           'increasing': {'color': 'green'}, 'decreasing': {'color': 'red'}},
    title = {"text": f"<b>Total Titles Released</b><br><span style='font-size:16px;color:gray'>Year: {titles_per_year['year_added'].iloc[0]}</span>"},
    number = {'font': {'size': 70, 'color': 'white'}},
    domain = {'x': [0, 1], 'y': [0, 1]}
)

#Creating frames
frames = []
for i in range(1, len(titles_per_year)):
    year = titles_per_year['year_added'].iloc[i]
    count = titles_per_year['total_titles'].iloc[i]
    ref = titles_per_year['total_titles'].iloc[i - 1]

    frame = go.Frame(
        data = [go.Indicator(
            mode = "number+delta",
            value = count,
            delta = {'reference': ref, 'relative': True, 'position': "right",
                   'increasing': {'color': 'green'}, 'decreasing': {'color': 'red'}},
            title = {"text": f"<b>Total Titles Released</b><br><span style='font-size:16px;color:gray'>Year: {year}</span>"},
            number = {'font': {'size': 70, 'color': 'white'}},
            domain = {'x': [0, 1], 'y': [0, 1]}
        )],
        name = str(year)
    )
    frames.append(frame)

#Steps for slider
steps = []
for i, year in enumerate(titles_per_year['year_added']):
    steps.append(dict(
        method = "animate",
        args = [[str(year)],
              {"frame": {"duration": 1000, "redraw": True},
               "mode": "immediate"}],
        label = str(year)
    ))

#Now constructing the figure
kpi_fig = go.Figure(
    data = [initial_trace],
    frames = frames,
    layout = go.Layout(
        template = 'plotly_dark',
        height = 300,
        margin = dict(t = 100, b = 0, l = 0, r = 0),
        updatemenus =[dict(
            type ="buttons",
            showactive =False,
            buttons =[
                dict(label= "Play",
                     method = "animate",
                     args = [None, {"frame": {"duration": 1000, "redraw": True},
                                  "fromcurrent": True, "transition": {"duration": 300}}]),
                dict(label = "Pause",
                     method = "animate",
                     args = [[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])
            ],
            direction = "left",
            pad = {"r": 10, "t": 70},
            x = 0.1,
            y = 0,
            xanchor = "right",
            yanchor = "top"
        )],
        sliders = [dict(
            active = 0,
            currentvalue = {"prefix": "Year: ", "font": {"size": 20}},
            pad = {"t": 50},
            steps = steps,
            x = 0.1,
            y = 0,
            len = 0.8,
            font = {"size": 16}
        )]
    )
)

kpi_fig.show()


*Adding Insights*

Over the years, Netflix’s content universe has evolved from predominantly U.S.-centric productions to a globally diverse portfolio. With a significant increase in content originating from countries like South Korea, India, Spain, and Germany, Netflix is clearly investing in regionally tailored stories. This strategic expansion not only caters to local audiences but also fuels global hits (e.g., Squid Game, Money Heist), showcasing Netflix’s ability to universalize regional narratives. The genre mix is also broadening—with documentaries, anime, reality shows, and international dramas contributing to a richer, more inclusive content landscape.

2. ***Inside Netflix :- A Deep Dive Through Animated Dashboards and Genre Dynamics***

In [ ]:
print(netflix_df.columns)


Animated the chart

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Copy and prepare data (adjust column names if different)
df = netflix_df.copy()

# Convert 'listed_in' string to list of genres
df['genre_list'] = df['listed_in'].str.split(', ')

# Ensure 'year_added' is numeric (extract year if datetime)
if not pd.api.types.is_numeric_dtype(df['year_added']):
    df['year_added'] = pd.to_datetime(df['year_added'], errors='coerce').dt.year

# Drop rows missing critical data
df = df.dropna(subset=['year_added', 'genre_list'])

# Extract first genre only for simplicity
df['primary_genre'] = df['genre_list'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 'Unknown')

# Group by year and genre to count titles
data = df.groupby(['year_added', 'primary_genre']).size().reset_index(name='count')

# Get sorted unique years and genres
years = sorted(data['year_added'].unique())
genres = sorted(data['primary_genre'].unique())

# Create a color palette for genres
color_map = px.colors.qualitative.Set2
colors = {genre: color_map[i % len(color_map)] for i, genre in enumerate(genres)}

# Create initial traces for the first year
initial_year = years[0]
fig = go.Figure()

for genre in genres:
    y_vals = data[(data['primary_genre'] == genre) & (data['year_added'] == initial_year)]['count']
    y = y_vals.values[0] if not y_vals.empty else 0
    fig.add_trace(go.Scatter(
        x = [initial_year],
        y = [y],
        mode = 'lines+markers',
        name = genre,
        line = dict(color = colors[genre], width = 3),
        marker = dict(size=6)
    ))

# Create animation frames for each year
frames = []
for year in years:
    frame_data = []
    for genre in genres:
        y_vals = data[(data['primary_genre'] == genre) & (data['year_added'] == year)]['count']
        y = y_vals.values[0] if not y_vals.empty else 0
        frame_data.append(go.Scatter(
            x = [year],
            y = [y],
            mode = 'lines+markers',
            line = dict(color = colors[genre], width = 3),
            marker = dict(size = 6)
        ))
    frames.append(go.Frame(data = frame_data, name = str(year)))

fig.frames = frames

# Update layout with dark theme and better spacing for buttons
fig.update_layout(
    title = dict(text = 'Netflix Genre Trends Over the Years', font = dict(color = 'white')),
    width = 1100,
    height = 600,
    paper_bgcolor = '#111111',
    plot_bgcolor = '#222222',
    xaxis = dict(
        title = 'Year',
        titlefont = dict(color = 'white'),
        tickfont = dict(color = 'lightgray'),
        linecolor = 'lightgray',
        gridcolor = 'gray',
        range = [min(years), max(years)],
        dtick = 2,
        tickangle = 45,
        tickvals = years[::2],
        ticktext = [str(y)[-2:] for y in years[::2]],
    ),
    yaxis = dict(
        title = 'Number of Titles',
        titlefont = dict(color = 'white'),
        tickfont = dict(color = 'lightgray'),
        linecolor = 'lightgray',
        gridcolor = 'gray',
        range = [0, data['count'].max() + 10]
    ),
    legend = dict(font = dict(color = 'white')),
    updatemenus = [dict(
        type = 'buttons',
        showactive = False,
        y = 1.05,          # moved buttons down so no overlap with title
        x = 0.1,
        xanchor = 'left',
        yanchor = 'top',
        buttons = [
            dict(label = 'Play',
                 method = 'animate',
                 args = [None, dict(frame = dict(duration = 700, redraw = True), fromcurrent = True)]),
            dict(label = 'Pause',
                 method = 'animate',
                 args = [[None], dict(frame = dict(duration = 0, redraw = False), mode = 'immediate')])
        ]
    )]
)

# Add slider for better control
sliders = [dict(
    active = 0,
    y = -0.1,
    x = 0.1,
    len = 0.9,
    currentvalue = dict(font = dict(color = 'white'), prefix = 'Year: ', visible = True, xanchor = 'right'),
    steps=[dict(method = 'animate',
                args = [[str(year)], dict(mode = 'immediate', frame = dict(duration = 700, redraw = True), transition = dict(duration = 300))],
                label = str(year)) for year in years]
)]

fig.update_layout(sliders=sliders)

fig.show()


Created the dashboard

In [ ]:
import pandas as pd
import plotly.express as px

# Sample Netflix-like genre-year data (replace this with your real data)
data = {
    'year_added': [2020, 2020, 2021, 2021, 2022, 2022, 2023, 2023],
    'genre': ['Drama', 'Comedy', 'Drama', 'Comedy', 'Drama', 'Comedy', 'Drama', 'Comedy'],
    'count': [100, 80, 120, 90, 130, 95, 125, 85]
}

df = pd.DataFrame(data)

fig = px.bar(df,
             x = 'count',
             y = 'genre',
             color = 'genre',
             animation_frame = 'year_added',
             orientation = 'h',
             title = 'Animated Netflix Genre Trends Over Years',
             color_discrete_sequence = px.colors.qualitative.Set2)

fig.update_layout(
    xaxis_title = 'Number of Titles',
    yaxis_title = 'Genre',
    paper_bgcolor = '#111',
    plot_bgcolor = '#222',
    font = dict(color = 'white'),
    height = 500,
    transition = {'duration': 800}
)

fig.show()


*Adding insights*

1. Dominance of Drama Genre
The Drama genre consistently stays at the top across all years.

This indicates that Netflix heavily invests in emotionally engaging and storyline-rich content, likely due to strong viewer retention and critical acclaim.

2. Steady Popularity of Comedy
Comedy maintains a stable position, often ranking second.

Lighter genres like comedy continue to attract global audiences, especially during times of stress (e.g., post-2020 pandemic surge in demand).

3. Narrowing Gap Between Top Genres
The gap between Drama and other genres slightly reduces after 2021, suggesting Netflix is trying to diversify its content portfolio.

Genres like Thriller, Action, and Romance begin to show consistent growth patterns.

4. Rise and Fall Patterns
Some genres see spikes in certain years — possibly tied to global events or successful titles (e.g., Korean drama boom post Squid Game).

You can observe how quickly viewer interests shift over just a couple of years.

5. Platform Strategy Reflection
Netflix’s strategic push to localize content is visible — lesser-known genres see small surges, which could reflect regional content (e.g., documentaries, anime, reality TV).

The animation reveals the global experimentation and risk-taking nature of Netflix's content strategy.



3. ***Visualizing Netflix: A Story of Global Content, Genres, and Trends***

*Creating the chart*

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

# Replace with your actual file path
netflix_df = pd.read_csv('netflix_titles.csv')

# Preview to confirm it's loaded correctly
netflix_df.head()


In [ ]:
import pandas as pd

netflix_df['release_year'] = pd.to_datetime(netflix_df['date_added'].str.strip(), errors = 'coerce')

netflix_df['genre'] = netflix_df['listed_in'].str.split(',')
netflix_exploded = netflix_df.explode('genre')

genre_trend = netflix_exploded.groupby(['release_year', 'genre']).size().reset_index(name='count')

stream_df = genre_trend.pivot(index = 'release_year', columns = 'genre', values = 'count').fillna(0)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

base = [0] * len(stream_df)

for genre in stream_df.columns:
   fig.add_trace(go.Scatter(
       x = stream_df.index,
       y = base,
       mode = 'lines',
       line = dict(width = 0),
       showlegend = False
   ))

   fig.add_trace(go.Scatter(
       x = stream_df.index,
       y = base + stream_df[genre].values,
       mode = 'lines',
       line = dict(width = 0.5),
       fill = 'tonexty',
       name = genre
   ))

   base += stream_df[genre].values


   fig.update_layout(
      title = 'Netflix Genre Stream Over Time',
      xaxis_title = 'Year',
      yaxis_title = 'Number of Titles',
      template = 'plotly_dark',
      showlegend = True,
      height = 600
  )

fig.show()

*Animating the chart*

In [ ]:
import plotly.graph_objects as go

#Filtering the top 3 genres
top_genres = stream_df.sum().sort_values(ascending = False).head(3).index
stream_df_top = stream_df[top_genres]
stream_df_reset = stream_df_top.reset_index()

#Creating the frames
frames = []
for i in range(1, len(stream_df_reset), 3):  #frames are being limited for clarity
    frame = go.Frame(
        data=[
            go.Scatter(
                x = stream_df_reset['release_year'][:i+1],
                y = stream_df_reset[genre][:i+1],
                mode = 'lines',
                stackgroup = 'one'
            )
            for genre in top_genres
        ],
        name = str(stream_df_reset['release_year'][i])
    )
    frames.append(frame)

#Creating the initial figure
fig = go.Figure(
    data=[
        go.Scatter(
            x = stream_df_reset['release_year'],
            y = stream_df_reset[genre],
            mode = 'lines',
            stackgroup = 'one',
            name = genre
        )
        for genre in top_genres
    ],
    frames = frames
)

#Updated about pause, play and silder
fig.update_layout(
    title = 'Netflix Genre Growth Over Time (Top 3 Genres)',
    xaxis_title = 'Year',
    yaxis_title = 'Number of Titles',
    template = 'plotly_dark',
    updatemenus = [dict(
        type = 'buttons',
        showactive = True,
        buttons = [
            dict(
                label = Play',
                method = 'animate',
                args = [None, dict(frame = dict(duration = 500, redraw = True), fromcurrent = True, mode = 'immediate')]
            ),
            dict(
                label = 'Pause',
                method = 'animate',
                args = [[None], dict(frame = dict(duration = 0, redraw = False), mode = 'immediate')]
            )
        ],
        direction = 'left',
        pad = {'r': 10, 't': 10},
        x = 0.1,
        y = 1.1
    )],
    sliders = [dict(
        steps = [dict(method = 'animate', args = [[f.name]], label = f.name) for f in frames],
        transition = dict(duration = 0),
        x = 0, y = 0,
        currentvalue = dict(prefix = "Year: ")
    )]
)

fig.show()


*Adding Dashboard*

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

netflix_df = pd.read_csv('netflix_titles.csv')

In [ ]:
import plotly.graph_objects as go
import pandas as pd

#sample data
years = [2015, 2016, 2017, 2018, 2019]
data = {
    'Drama': [10, 15, 20, 25, 30],
    'Comedy': [5, 10, 15, 10, 5],
    'Action': [2, 4, 6, 8, 10]
}
df = pd.DataFrame(data, index=years)
top_genres = df.columns.tolist()

#Intial Data
initial_data = [
    go.Scatter(
        x = [years[0]],
        y = [df.loc[years[0], genre]],
        mode = 'lines',
        stackgroup = 'one',
        name = genre
    ) for genre in top_genres
]

#Frames are implemented
frames = []
for i, year in enumerate(years):
    frame_data = [
        go.Scatter(
            x = years[:i+1],
            y = df[genre].iloc[:i+1],
            mode ='lines',
            stackgroup ='one',
            name = genre
        )
        for genre in top_genres
    ]
    frames.append(go.Frame(data = frame_data, name = str(year)))

#Layout of the chart
layout = go.Layout(
    title = "Genre Trends Over Time",
    plot_bgcolor = 'black',
    paper_bgcolor = 'black',
    font = dict(color = 'white'),
    xaxis = dict(
        showgrid = False,
        zeroline = False,
        color = 'white',
        showline = True,
        linecolor = 'white'
    ),
    yaxis = dict(
        showgrid = False,
        zeroline = False,
        color = 'white',
        showline = True,
        linecolor = 'white'
    ),
    updatemenus = [{
        'type': 'buttons',
        'buttons': [
            {
                'label': 'Play',
                'method': 'animate',
                'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}]
            },
            {
                'label': 'Pause',
                'method': 'animate',
                'args': [[None], {'frame': {'duration': 0}, 'mode': 'immediate'}]
            }
        ]
    }],
    sliders = [{
        'steps': [{
            'method': 'animate',
            'args': [[str(year)], {'mode': 'immediate', 'frame': {'duration': 300}}],
            'label': str(year)
        } for year in years]
    }]
)

fig = go.Figure(data = initial_data, layout = layout, frames = frames)
fig.show()


*Adding Insights*

1. Drama is the star of the show.
You can see it steadily climbing year after year. Netflix seems to be doubling down on dramas, and honestly, people just can’t get enough of those gripping storylines.

2. Comedy’s rollercoaster ride.
Comedy had a good run early on but then took a dip. Maybe the laughs got a little less frequent, or Netflix shifted focus — who knows? But it’s interesting to watch it rise and fall like that.

3. Action is quietly gaining ground.
Not as flashy as drama, but there’s a steady, slow climb in action titles. Maybe folks are craving a bit more adrenaline these days.

4. What’s the big picture?
Drama dominates, but Netflix is clearly experimenting — they’re not putting all their eggs in one basket. It’s like a buffet: something for everyone, but some dishes (genres) get more attention.

5. If you’re a content strategist...
Drama is where the magic’s happening, so maybe pump more resources there. But keep an eye on action — it’s a sleeper hit that might blow up soon.



4. ***The Evolution Of Netflix: Animated Insights and Dashboard Analysis***

*Creating the chart*

In [ ]:
import pandas as pd
import plotly.graph_objects as go
#Changing 'date_added' and 'release_year' to datetime
netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'], errors = 'coerce')
netflix_df['release_year'] = netflix_df['date_added'].dt.year

#Group by year and type (Movie or TV Show)
format_trend = netflix_df.groupby(['release_year','type']).size().reset_index(name = 'count')

#Pivot so we can get separate columns
format_pivot = format_trend.pivot(index = 'release_year', columns = 'type', values = 'count').fillna(0).astype(int)

#Ensuring all relevant years are covered
format_pivot = format_pivot.loc[format_pivot.index.notnull()]
years = format_pivot.index.tolist()

In [ ]:
#Intialising the figure
initial_data = [
    go.Scatter(x = [years[0]], y = [format_pivot['Movie'].iloc[0]], mode = 'lines', name = 'Movies', line = dict(color = 'red')),
    go.Scatter(x = [years[0]], y = [format_pivot['TV Show'].iloc[0]], mode = 'lines', name = 'TV Shows', line = dict(color = 'cyan'))
]

frames = []

for i in range(1, len(years)):
  frame = go.Frame(
      data = [
          go.Scatter(x = years[:i+1], y = format_pivot['Movie'].iloc[:i+1], mode = 'lines', name = 'Movies', line = dict(color = 'red')),
          go.Scatter(x = years[:i+1], y = format_pivot['TV Show'].iloc[:i+1], mode = 'lines', name = 'TV Shows', line = dict(color = 'cyan')),
      ],
      name = str(years[i])
  )
  frames.append(frame)

#Layout with black theme
layout = go.Layout(
    title = 'Movies vs TV Shows on Netflix (Animated)',
    plot_bgcolor = 'black',
    paper_bgcolor = 'black',
    font = dict(color = 'white'),
    xaxis = dict(title = 'Year', color = 'white'),
    yaxis = dict(title = 'Number of Titles', color = 'white'),
    updatemenus = [{
        'type': 'buttons',
        'buttons': [
            {'label': 'Play', 'method': 'animate', 'args':[None, {'frame':{'duration': 500}, 'fromcurrent': True}]},
            {'label': 'Pause', 'method': 'animate', 'args':[[None], {'frame':{'duration': 0}, 'mode': 'immediate'}]}

        ]
    }],
    sliders = [{
        'steps': [{
            'method': 'animate',
            'args': [[str(year)], {'mode': 'immediate', 'frame': {'duration':300}}],
        } for year in years]
    }]
)

#Final animated figure
fig = go.Figure(data = initial_data, layout = layout, frames = frames)
fig.show()

In [ ]:
total_titles = len(netflix_df)
total_movies = len(netflix_df[netflix_df['type'] == 'Movie'])
total_shows = len(netflix_df[netflix_df['type'] == 'TV Show'])

fig.add_annotation(
    xref = 'paper', yref = 'paper',
    x = 0.5, y = 1.1,
    showarrow = False,
    font = dict(size = 14, color = 'white'),
    align = 'center',
    text = f"<b>Total Titles:</b> {total_titles} | <b>Movies:</b> {total_movies} | <b>TV Shows:</b> {total_shows}"
)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

#Preparing the datasets
netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'], errors = 'coerce')
netflix_df['genre'] = netflix_df['listed_in'].str.split(',')
netflix_exploded = netflix_df.explode('genre')
netflix_exploded['genre'] = netflix_exploded['genre'].str.strip()
netflix_exploded['release_year'] = pd.DatetimeIndex(netflix_exploded['date_added']).year
netflix_exploded = netflix_exploded.dropna(subset=['release_year'])

#Group and Pivot
genre_trend = netflix_exploded.groupby(['release_year', 'genre']).size().reset_index(name = 'count')
pivot_df = genre_trend.pivot(index = 'release_year', columns = 'genre', values = 'count').fillna(0)

#Top genres
top_genres = pivot_df.sum().sort_values(ascending = False).head(5).index.tolist()
stream_df = pivot_df[top_genres]
years = stream_df.index.tolist()

#Initial data for first frame
initial_data = [
    go.Scatter(
        x = [years[0]],
        y = [stream_df.loc[years[0], genre]],
        mode = 'lines',
        stackgroup = 'one',
        name = genre
    ) for genre in top_genres
]

#Creating animation frames
frames = []
for i, year in enumerate(years):
    frame_data = [
        go.Scatter(
            x = years[:i+1],
            y = stream_df[genre].iloc[:i+1],
            mode = 'lines',
            stackgroup = 'one',
            name = genre
        ) for genre in top_genres
    ]
    frames.append(go.Frame(data = frame_data, name = str(year)))

#Definign Layout
layout = go.Layout(
    title = "Netflix Genre Trends Over Time (Animated)",
    plot_bgcolor = 'black',
    paper_bgcolor = 'black',
    font = dict(color = 'white'),
    xaxis = dict(
        title = 'Year',
        color = 'white',
        showgrid = False,
        linecolor = 'white'
    ),
    yaxis = dict(
        title = 'Number of Titles',
        color = 'white',
        showgrid = False,
        linecolor = 'white'
    ),
    updatemenus = [{
        'type': 'buttons',
        'y': 0.85,
        'x': 0.1,
        'xanchor': 'left',
        'yanchor': 'top',
        'buttons': [
            {
                'label': 'Play',
                'method': 'animate',
                'args': [None, {
                    'frame': {'duration': 500, 'redraw': True},
                    'fromcurrent': True
                }]
            },
            {
                'label': 'Pause',
                'method': 'animate',
                'args': [[None], {
                    'frame': {'duration': 0},
                    'mode': 'immediate'
                }]
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }],
    sliders = [{
        'y': -0.05,
        'x': 0.1,
        'len': 0.8,
        'pad': {'b': 10, 't': 50},
        'steps': [{
            'method': 'animate',
            'args': [[str(year)], {'mode': 'immediate', 'frame': {'duration': 300}}],
            'label': str(year)
        } for year in years]
    }]
)

#Finally building the chart
fig = go.Figure(data = initial_data, layout = layout, frames = frames)
fig.show()


*Adding Insights*

1. Dramatic Rise of Drama
Drama consistently remains the most added genre over the years. Its steady rise suggests it's a cornerstone of Netflix’s content strategy, likely due to its broad appeal across age groups.

2. Comedy Comes and Goes
While Comedy had a strong presence initially, its trend shows some fluctuations. This could indicate experimentation with different styles or changing viewer preferences.

3. Action Gaining Ground
Action titles have shown a clear upward trend — possibly reflecting Netflix’s growing investment in original action series and international thrillers.

4. Shift Toward Genre Diversity
Starting from around 2018, the variety in genre content seems to expand. Netflix is likely responding to global audience demand by diversifying beyond traditional genres.



 5. ***Netflix Analytics Dashboard: Genre Growth, Global Reach, and Time Trends***

*Creating the chart*

In [ ]:
import plotly.express as px
import pandas as pd

data = {
    'country' : ['United States'] * 4 + ['United Kingdom'] * 4 + ['India'] * 4 + ['Canada'] * 4 + ['Germany'] * 4,
    'year' : [2018, 2019, 2020, 2021] * 5,
    'titles_count': [500, 550, 600, 650, 100, 120, 140, 160, 200, 250, 300, 350, 150, 180, 210, 240, 80, 100, 120, 140]
}

df = pd.DataFrame(data)

fig = px.choropleth(df,
                    locations = 'country',
                    locationmode = 'country names',
                    color = 'titles_count',
                    animation_frame = 'year',
                    color_continuous_scale = 'plasma',
                    title = 'Netflix Titles by Country Over Time',
                    template = 'plotly_dark'
)
fig.update_layout(
    geo = dict(bgcolor = 'rgba(0,0,0,0)'),
    paper_bgcolor = 'black',
    plot_bgcolor = 'black'
)
fig.show()

*Adding Dashboard*

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Calculating KPIs
total_titles = len(netflix_df)
total_movies = len(netflix_df[netflix_df['type'] == 'Movie'])
total_shows = len(netflix_df[netflix_df['type'] == 'TV Show'])

#Creating rows and columns
fig = make_subplots(rows = 1, cols = 3, specs = [[{"type": "indicator"}]*3])

fig.add_trace(go.Indicator(
    mode = "number",
    value = total_titles,
    title = {"text": "Total Titles"},
    number = {"font": {"size": 36}},
), row = 1, col = 1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = total_movies,
    title = {"text": "Movies"},
    number = {"font": {"color": "#EF553B", "size": 36}},
), row = 1, col = 2)

fig.add_trace(go.Indicator(
    mode = "number",
    value = total_shows,
    title = {"text": "TV Shows"},
    number = {"font": {"color": "#00CC96", "size": 36}},
), row = 1, col = 3)

fig.update_layout(
    paper_bgcolor = "black",
    plot_bgcolor = "black",
    font = {"color": "white", "family": "Arial"},
    height = 220
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

#Extracting and Expanding
genre_df = netflix_df.copy()
genre_df = genre_df.explode('listed_in'.split(', ') if isinstance(genre_df['listed_in'].iloc[0], str) else genre_df['listed_in'])
genre_df['genre'] = genre_df['listed_in'].str.split(', ')
genre_df = genre_df.explode('genre')
genre_df['genre'] = genre_df['genre'].str.strip()

#Group by and year
genre_counts = genre_df.groupby(['release_year', 'genre']).size().reset_index(name='count')
genre_counts = genre_counts[genre_counts['release_year'] >= 2008]  # Clean years for animation

#Race style animated bar chart
fig = px.bar(
    genre_counts.sort_values(by = "count", ascending = False),
    x = "count", y = "genre",
    color = "genre",
    animation_frame = "release_year",
    orientation = "h",
    range_x = [0, genre_counts["count"].max() + 20],
    title = "Genre Growth Over Time (2008+)",
    height = 600,
    template = "plotly_dark"
)

fig.update_layout(
    showlegend = False,
    xaxis_title = "Number of Titles",
    yaxis_title = "Genre",
    font = {"family": "Arial"},
    title_font_size = 24
)

fig.show()


*Adding Insights*

1. Drama is the most dominant and consistent genre on Netflix over the years, reflecting its popularity with viewers.

2. Documentaries and Comedies show a steady upward trend, indicating increasing interest in informative and light-hearted content.

3. After 2015, there is a noticeable surge in Thriller and Horror genres, likely due to changing viewer tastes and Netflix’s expanded original content in these categories.

4. Some niche genres such as Romance or Action & Adventure appear irregularly but maintain a steady presence, suggesting focused but smaller audience segments.

5. Overall, Netflix’s genre portfolio has diversified over time, with growth in multiple genres reflecting a broad content strategy.



6. ***Netflix Reimagined: A Visual Story of Streaming Content Through Time***

*Creating the chart*

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

data = {
    'country': ['USA','India','UK','USA','South Korea','India','UK','USA'],
    'type': ['Movie','Movie','TV Show','TV Show','TV Show','Movie','Movie','Movie'],
    'genre': ['Drama','Action','Documetary','Comedy','Romance','Horror','Comedy','Thriller'],
    'release_year': [2019, 2020, 2019, 2021, 2022, 2020, 2021, 2023],
    'duration': [100, 110, 50, 45, 60, 90, 85, 105]
}

df = pd.DataFrame(data)
df.head()

In [ ]:
!pip install plotly --upgrade


In [ ]:
import pandas as pd
import plotly.graph_objects as go

#Sample dataset
data = {
    'country': ['USA', 'India', 'UK', 'USA', 'South Korea', 'India', 'UK', 'USA'],
    'type': ['Movie', 'Movie', 'TV Show', 'TV Show', 'TV Show', 'Movie', 'Movie', 'Movie'],
    'genre': ['Drama', 'Action', 'Documentary', 'Comedy', 'Romance', 'Horror', 'Comedy', 'Thriller'],
    'release_year': [2019, 2020, 2019, 2021, 2022, 2020, 2021, 2023],
    'duration': [100, 110, 50, 45, 60, 90, 85, 105]
}
df = pd.DataFrame(data)

#Unique nodes
nodes = pd.unique(df[['country', 'type', 'genre']].values.ravel())
node_indices = {label: i for i, label in enumerate(nodes)}


In [ ]:
frames = []
#Created an empty list
years = sorted(df['release_year'].unique())
#Getting a sorted list

#Loop through each year to create individual frames
for year in years:
    df_year = df[df['release_year'] == year]

    #Counting connections Type (Movie/TV Shows)
    ct = df_year.groupby(['country', 'type']).size().reset_index(name = 'count')

    #Counting connections Type (Genre)
    tg = df_year.groupby(['type', 'genre']).size().reset_index(name = 'count')

    #Merging both dataframes
    links_df = pd.concat([
        ct.rename(columns = {'country': 'source', 'type': 'target'}),
        tg.rename(columns = {'type': 'source', 'genre': 'target'})
    ])
    #Mapping source and target labels
    sources = links_df['source'].map(node_indices)
    targets = links_df['target'].map(node_indices)
    values = links_df['count']

    #Building a Sankey chart
    sankey_frame = go.Sankey(
        node = dict(pad = 15, thickness = 20, line = dict(color = "black", width = 0.5), label = list(nodes)),
        link = dict(source = sources, target = targets, value = values)
    )

    #Adding this frame
    frames.append(dict(data = [sankey_frame], name=str(year)))

initial_data = frames[0]['data'][0]

#Creating the final animated figure
fig = go.Figure(
    data = [initial_data],
    frames = frames,
    layout = go.Layout(
        title_text = "Netflix Animated Flow",
        font = dict(size = 12),
        height = 600,
        paper_bgcolor = '#000000',
        plot_bgcolor = '#000000',
        updatemenus=[dict(
    type = "buttons",
    showactive = False,
    buttons = [
        dict(
            label = " Play",
            method = "animate",
            args = [None, {
                "frame": {"duration": 1500, "redraw": True},
                "fromcurrent": True,
                "transition": {"duration": 300}
            }]
        ),
        dict(
            label = "Pause",
            method = "animate",
            args = [[None], {
                "mode": "immediate",
                "frame": {"duration": 0, "redraw": False},
                "transition": {"duration": 0}
            }]
        )
    ]
)],

        sliders=[dict(steps = [dict(method = 'animate', args = [[str(y)], dict(mode = 'immediate',
                        frame = dict(duration = 1500, redraw = True), transition=  dict(duration = 300))],
                        label = str(y)) for y in years], x = 0.1, y = 0,
                        currentvalue = dict(font = dict(size = 14), prefix = " Year: "), len = 0.9)]
    )
)

fig.show()


*Adding Dashboard*

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Grouping the data by release year and genre
genre_year = df.groupby(['release_year','genre']).size().reset_index(name = 'count')
years = sorted(genre_year['release_year'].unique())
genres = genre_year['genre'].unique()

theta = np.linspace(0, 360, len(genres), endpoint = False)
frames = []

# Create animation frames for each year
for year in years:
    data_year = genre_year[genre_year['release_year'] == year]

    genre_counts = [data_year[data_year['genre'] == g]['count'].values[0] if g in data_year['genre'].values else 0 for g in genres]

    frame = go.Barpolar(
        r = genre_counts,
        theta = theta,
        width = [360/len(genres)]*len(genres),
        marker = dict(
            color = genre_counts,
            colorscale = 'Viridis',
            line = dict(color = 'white', width = 1)
        ),
        opacity = 0.8,
        name = str(year)
    )

    frames.append(go.Frame(data = [frame], name = str(year)))

# Use first year's data as initial plot
initial_data = frames[0].data

# Create animated circular bar chart
fig = go.Figure(
    data = initial_data,
    layout = go.Layout(
        title = 'Netflix Genre Spread Over Time',
        template = 'plotly_dark',
        polar = dict(
            bgcolor = '#000000',
            radialaxis = dict(visible = True, showticklabels = False, ticks = ''),
            angularaxis  = dict(showticklabels = True, ticks = '')
        ),
        paper_bgcolor = '#000000',
        font = dict(color = 'white'),

        # Add play and pause buttons
        updatemenus = [dict(
            type = 'buttons',
            showactive = False,
            buttons = [
                dict(label  = 'Play', method = 'animate', args = [None, {
                    'frame': {'duration': 1200, 'redraw': True},
                    'fromcurrent': True,
                    'transition': {'duration': 300}
                }]),
                dict(label = 'Pause', method = 'animate', args = [[None], {
                    'mode'      : 'immediate',
                    'frame'     : {'duration': 0, 'redraw': False},
                    'transition': {'duration': 0}
                }])
            ]
        )],

        # Slider for year selection
        sliders = [dict(
            steps = [dict(method = 'animate',
                                  args = [[str(y)], dict(mode = 'immediate',
                                                           frame = dict(duration = 1200, redraw = True),
                                                           transition = dict(duration = 300))],
                                  label  = str(y)) for y in years],
            currentvalue  = dict(prefix = 'Year: ', font = dict(color = 'white')),
            x = 0.1,
            y = 0,
            len = 0.9
        )]
    ),
    frames = frames
)

fig.show()


*Adding insights*

1. Consistent Dominance of Drama:
Drama remains the most prominent genre across nearly all years. Its broad appeal and adaptability across cultures and formats make it a cornerstone of Netflix's content library.

2. Notable Growth in Documentaries (Post-2017):
Starting around 2017, there's a significant uptick in documentary content. This aligns with Netflix's increased investment in non-fiction storytelling, often centered around real-world issues and in-depth explorations.

3. Comedy's Resurgence (2019–2020):
While Comedy showed a relatively stable presence in earlier years, it witnessed a noticeable increase in later years — likely driven by the rising popularity of stand-up specials and lighthearted series during the pandemic period.

4. Seasonal Surges in Thriller and Horror:
Although not dominant year-round, Thriller and Horror genres exhibit periodic spikes, suggesting Netflix’s targeted release strategy during specific times of the year (e.g., Halloween).

5. Steady Contribution of Family-Oriented Content:
Genres focused on Children and Family maintain a consistent, though modest, presence — indicating a steady commitment to producing content suitable for all age groups.

6. Increased Genre Diversification in Recent Years:
From 2020 onward, the platform demonstrates a broader genre mix, pointing to Netflix’s expansion into regional content, experimental formats, and niche categories as part of its global growth strategy.

7.***Unpacking Netflix: Animated Visuals, Trends, and Global Content Strategy***

*Creating the chart*

In [ ]:
print(df.columns)


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

# Load your CSV file into a DataFrame
df = pd.read_csv('netflix_titles.csv')

# Preview the first few rows to confirm
df.head()


In [ ]:
import pandas as pd


df = pd.read_csv('netflix_titles.csv')


print(df.columns)


In [ ]:
import pandas as pd

data = {
    'country': ['USA', 'India', 'UK', 'USA', 'India', 'UK', 'USA', 'India', 'UK'],
    'year': [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022],
    'titles_count': [500, 300, 200, 550, 350, 250, 600, 400, 300]
}

df = pd.DataFrame(data)


In [ ]:
import plotly.io as pio
pio.renderers.default = 'colab'


In [ ]:
!pip install plotly


In [ ]:
!pip install plotly --upgrade
import plotly.io as pio
pio.renderers.default = 'colab'


In [ ]:
import pandas as pd
import plotly.express as px

#Sample data (since actual data is too much and it gets tough for the server to handle)
df = pd.DataFrame({
    'country': ['USA', 'India', 'UK', 'USA', 'India', 'UK', 'USA', 'India', 'UK'],
    'year': [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022],
    'titles_count': [400, 300, 200, 450, 350, 250, 500, 400, 300]
})

fig = px.scatter(
    df,
    x = 'year',
    y = 'titles_count',
    animation_frame = 'year',
    animation_group = 'country',
    size = 'titles_count',
    color='country',
    hover_name = 'country',
    size_max = 60,
    range_x = [2019, 2023],
    range_y = [0, df['titles_count'].max() + 100],
    title = 'Netflix Titles Count Growth by Country',
    template = 'plotly_dark'
)

fig.update_layout(
    xaxis_title = 'Year',
    yaxis_title = 'Titles Count',
    paper_bgcolor = 'black',
    font = dict(color = 'white')
)

fig.show()


*Creating Dashboard*

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML

#Sample data
df = pd.DataFrame({
    'country': ['USA', 'India', 'UK', 'France', 'Germany', 'USA', 'India', 'UK', 'France', 'Germany', 'USA', 'India', 'UK', 'France', 'Germany'],
    'year': [2020]*5 + [2021]*5 + [2022]*5,
    'titles_count': [400, 300, 200, 150, 100, 450, 350, 250, 180, 120, 500, 400, 300, 200, 150]
})

latest_year = df['year'].max()
countries = df['country'].unique()

#KPI
total_titles_latest = df[df['year'] == latest_year]['titles_count'].sum()

#Bubble chart
bubble_fig = px.scatter(
    df,
    x = 'year',
    y = 'titles_count',
    animation_frame = 'year',
    animation_group = 'country',
    size = 'titles_count',
    color = 'country',
    hover_name = 'country',
    size_max = 60,
    range_x = [2019, 2023],
    range_y = [0, df['titles_count'].max() + 100],
    title = 'Netflix Titles Growth by Country',
    template = 'plotly_dark'
)
bubble_fig.update_layout(
    xaxis_title = 'Year',
    yaxis_title = 'Titles Count',
    paper_bgcolor = 'black',
    font = dict(color = 'white')
)

#Bar Chart for the top 5
top5 = df[df['year'] == latest_year].sort_values('titles_count', ascending=False).head(5)

bar_fig = go.Figure(go.Bar(
    x = top5['country'],
    y = top5['titles_count'],
    marker_color = 'cyan',
    text = top5['titles_count'],
    textposition = 'auto'
))
bar_fig.update_layout(
    title = f'Top 5 Countries by Titles Count in {latest_year}',
    template = 'plotly_dark',
    paper_bgcolor = 'black',
    font = dict(color = 'white'),
    yaxis = dict(title = 'Titles Count'),
    xaxis = dict(title = 'Country')
)

#KPI Card in Plotly Indicator
kpi_fig = go.Figure(go.Indicator(
    mode = "number",
    value = total_titles_latest,
    title = {"text": f"Total Titles in {latest_year}"},
    number = {'font': {'size': 48, 'color': 'cyan'}},
    domain = {'x': [0, 1], 'y': [0, 1]}
))
kpi_fig.update_layout(
    paper_bgcolor = 'black',
    font = dict(color = 'white')
)

#Displaying the charts side by side
from plotly.subplots import make_subplots

dashboard = make_subplots(
    rows = 1, cols = 3,
    column_widths = [0.5, 0.25, 0.25],
    specs = [[{"type": "scatter"}, {"type": "indicator"}, {"type": "bar"}]],
    subplot_titles = ("Animated Titles Growth", "KPI", "Top Countries")
)

#Adding Bubble Chart
for trace in bubble_fig.frames[0].data:
    dashboard.add_trace(trace, row = 1, col = 1)

#Adding KPI
dashboard.add_trace(kpi_fig.data[0], row = 1, col = 2)

#Adding Bar Chart
for trace in bar_fig.data:
    dashboard.add_trace(trace, row=1, col=3)

dashboard.update_layout(
    title_text = "Netflix Content Dashboard",
    template = 'plotly_dark',
    paper_bgcolor = 'black',
    font = dict(color = 'white'),
    showlegend = True,
    height = 600
)

#Adding animation frames
dashboard.frames = bubble_fig.frames

#Adding updatemenus for play and pause
dashboard.update_layout(
    updatemenus = [dict(
        type = "buttons",
        showactive = False,
        y = 0.5,
        x = 1.05,
        xanchor = "left",
        yanchor = "top",
        buttons = [
            dict(label = "Play",
                 method = "animate",
                 args = [None, {"frame": {"duration": 1000, "redraw": True},
                              "fromcurrent": True, "transition": {"duration": 300}}]),
            dict(label = "Pause",
                 method = "animate",
                 args = [[None], {"frame": {"duration": 0, "redraw": False},
                                "mode": "immediate",
                                "transition": {"duration": 0}}])
        ]
    )]
)



dashboard.show()


*Adding Insights*

1. Netflix’s Global Growth Journey
Over the years, Netflix has transformed from a region-focused platform into a truly global content powerhouse. The animation clearly reflects this evolution, showing more countries contributing content each year.

2. Consistent Leaders in Content Production
Countries like the United States, India, and the United Kingdom remain steady leaders in content volume, highlighting their strong role in shaping Netflix’s library.

3. Rapid Surge in Content Volume
There's a noticeable spike in titles released annually, especially after 2016. This aligns with Netflix’s strategy to scale content production and cater to a growing international audience.

4. Shift Toward Diverse Storytelling
The visualisation brings to light Netflix’s increasing investment in content from non-Western regions. We see stronger representation from Asian, African, and Latin American countries as time progresses.

5. Phases of Strategic Expansion

* Early Years: Predominantly U.S.-based content

* Mid-2010s: Focused expansion across key markets

* Recent Years: Widespread global production with region-specific investments

6. Emerging Regional Voices
The rise in content from countries like South Korea, Turkey, and Mexico reflects Netflix’s commitment to regional storytelling and its effort to tap into diverse viewer bases worldwide.



8. ***Netflix in Motion: Dashboard-Driven Insights Across Countries and Categories***

*KPIs*

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_titles,
    title = {'text': 'Total Titles'},
    domain = {'x': [0.0, 0.22], 'y': [0, 1]}
))

fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_countries,
    title = {'text': 'Total Countries'},
    domain = {'x': [0.26, 0.48], 'y': [0, 1]}
))

fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_genres,
    title = {'text': 'Total Genres'},
    domain = {'x': [0.52, 0.74], 'y': [0, 1]}
))

fig.add_trace(go.Indicator(
    mode = 'number',
    value = peak_year,
    title = {'text': 'Peak Year'},
    domain = {'x': [0.78, 1.0], 'y': [0, 1]}
))

fig.update_layout(
    template = 'plotly_dark',
    paper_bgcolor = 'black',
    font = {'color': 'white', 'size': 14},
    height = 200,
    width = 900,  # wider width to avoid crowding
    margin = {'t': 30, 'b': 10}
)

fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

#Sample Netflix
netflix_df = pd.DataFrame({
    'title': ['Stranger Things', 'The Crown', 'Money Heist', 'Dark', 'Sacred Games',
              'Lupin', 'Narcos', 'Delhi Crime', 'Squid Game', 'Kingdom'],
    'country': ['USA', 'UK', 'Spain', 'Germany', 'India', 'France', 'Colombia', 'India', 'South Korea', 'South Korea'],
    'genre': ['Sci-Fi', 'Drama', 'Action', 'Thriller', 'Crime', 'Mystery', 'Crime', 'Crime', 'Thriller', 'Horror'],
    'year': [2016, 2016, 2017, 2017, 2018, 2021, 2015, 2019, 2021, 2019]
})

#Key Metrics
total_titles = netflix_df['title'].nunique()
total_countries = netflix_df['country'].nunique()
total_genres = netflix_df['genre'].nunique()
peak_year = netflix_df['year'].value_counts().idxmax()

#Subplot layout for 2x2 grid and 1 bar chart row
fig = make_subplots(
    rows = 3, cols = 2,
    specs = [
        [{'type': 'indicator'}, {'type': 'indicator'}],
        [{'type': 'indicator'}, {'type': 'indicator'}],
        [{'type': 'xy', 'colspan': 2}, None]
    ],
    row_heights = [0.22, 0.22, 0.56],
    vertical_spacing = 0.12
)

#Indicator cards
fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_titles,
    title = {'text': '<b>Total Titles</b>'},
    number = {'font': {'size': 36}}
), row = 1, col = 1)

fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_countries,
    title = {'text': '<b>Total Countries</b>'},
    number = {'font': {'size': 36}}
), row = 1, col = 2)

fig.add_trace(go.Indicator(
    mode = 'number',
    value = total_genres,
    title = {'text': '<b>Total Genres</b>'},
    number = {'font': {'size': 36}}
), row = 2, col = 1)

fig.add_trace(go.Indicator(
    mode = 'number',
    value = peak_year,
    title = {'text': '<b>Peak Year</b>'},
    number = {'font': {'size': 36}}
), row = 2, col = 2)

#Bar Chart
titles_per_year = netflix_df.groupby('year').size().reset_index(name = 'count')
fig.add_trace(go.Bar(
    x = titles_per_year['year'],
    y = titles_per_year['count'],
    marker = dict(color = 'lightskyblue', line = dict(color = 'white', width = 1)),
    hoverinfo = 'x+y',
    opacity = 0.85
), row = 3, col = 1)

#Updating Layout
fig.update_layout(
    title = '<b>Netflix Global Content KPIs and Yearly Trend</b>',
    template = 'plotly_dark',
    paper_bgcolor = 'black',
    plot_bgcolor = 'black',
    font = {'color': 'white', 'size': 14},
    height = 720,
    margin = {'t': 70, 'b': 60},
    xaxis3 = dict(
        title = 'Year',
        tickmode = 'linear',
        tickangle = -45,
        tickfont = dict(size = 12),
        showline = True,
        linewidth = 1,
        linecolor = 'white'
    ),
    yaxis3 = dict(
        title = 'Number of Titles',
        tickfont = dict(size = 12),
        showline = True,
        linewidth = 1,
        linecolor = 'white'
    )
)

fig.show()


*Adding Dashboard*

In [ ]:
import pandas as pd
import plotly.express as px

#Sample data
netflix_df = pd.DataFrame({
    'title': ['Stranger Things', 'The Crown', 'Money Heist', 'Dark', 'Sacred Games',
              'Lupin', 'Narcos', 'Delhi Crime', 'Squid Game', 'Kingdom'],
    'country': ['USA', 'UK', 'Spain', 'Germany', 'India', 'France', 'Colombia', 'India', 'South Korea', 'South Korea'],
    'genre': ['Sci-Fi', 'Drama', 'Action', 'Thriller', 'Crime', 'Mystery', 'Crime', 'Crime', 'Thriller', 'Horror'],
    'year': [2016, 2016, 2017, 2017, 2018, 2021, 2015, 2019, 2021, 2019]
})
print(netflix_df['genre'].unique())
print(netflix_df.groupby(['year', 'country', 'genre']).size())

more_data = pd.DataFrame({
    'title': ['Example Show']*30,
    'country': ['USA', 'India', 'UK']*10,
    'genre': ['Drama', 'Action', 'Horror']*10,
    'year': [2017, 2018, 2019]*10
})
netflix_df = pd.concat([netflix_df, more_data], ignore_index=True)

#Group by year, country, and genre to count titles
grouped = netflix_df.groupby(['year', 'country', 'genre']).size().reset_index(name = 'title_count')

#Bubble Chart
fig = px.scatter(
    grouped,
    x = 'country',
    y = 'genre',
    size = 'title_count',
    color = 'genre',
    animation_frame = 'year',
    size_max = 40,
    title = 'Netflix Titles by Country and Genre Over Time',
    labels = {'title_count': 'Number of Titles'},
    template = 'plotly_dark',
)

#Improved layout to avoid clustering
fig.update_layout(
    yaxis = {'categoryorder': 'total ascending'},
    xaxis_title = 'Country',
    yaxis_title = 'Genre',
    legend_title_text = 'Genre',
    height = 600,
    margin = dict(t = 60, b = 50)
)

fig.show()


*Adding Insights*

1. Dynamic Growth Across Regions: The animated visualization reveals how Netflix’s content library has evolved differently across countries over the years, reflecting localized content strategies and market demands.

2. Genre Popularity Varies by Region: We observe that genres like Crime and Thriller show strong representation across multiple countries, while others such as Sci-Fi or Mystery may be more concentrated in specific markets like the USA or France.

3. Expansion in Emerging Markets: Countries like India and South Korea display increasing title counts in recent years, illustrating Netflix’s strategic focus on tapping into diverse, growing audiences with regionally relevant genres.

4. Content Diversity Trends: The spread of genres across countries highlights Netflix’s commitment to offering a broad range of entertainment options — from drama and action to horror and sci-fi — tailored to varied viewer preferences globally.

5. Peak Activity Periods: The animation timeline also helps identify peak years for content additions, enabling us to correlate with global trends, licensing deals, or original production pushes that influenced the catalog size.

